In [ ]:
import re
from functools import partial

from tqdm.notebook import tqdm

import pandas as pd
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import levene
import matplotlib.pyplot as plt
import seaborn as sns

topics = {
    'a': 'Advice',
    'e': 'Entertainment',
    'g': 'Gossip',
    'i': 'Informational',
    'r': 'Recommendation',
    's': 'Social'
}

In [ ]:
df = pd.read_csv('data.csv', header=0, skiprows=[1,2], usecols=partial(re.fullmatch, r'[aegirs][123][+=-] (tone|clarity|intent)'))

display(df)
cols = df.columns

In [ ]:
df.columns = pd.MultiIndex.from_arrays([
    cols.map(lambda col: col[4:]),
    cols.map(lambda col: col[2]),
    cols.map(lambda col: col[0]),
    cols.map(lambda col: col[1]),
])

display(df)

In [ ]:
stacked = df.stack(level=[1,2,3])
stacked.index = stacked.index.droplevel(0).set_names(['label', 'topic', 'message'])
stacked = stacked.reset_index()
display(stacked)

In [ ]:
# import re

tukey_results = {}

for question in ['tone', 'clarity', 'intent']:
    tukey_results[question] = pairwise_tukeyhsd(endog=stacked[question], groups=stacked['label'], alpha=0.05)
    
    print(question, '\n', tukey_results[question].summary(), '\n\n')
    
    _ = tukey_results[question].plot_simultaneous(figsize=(5,3), xlabel=f'{question.capitalize()} ranking', ylabel='Label')
    plt.title(f'Pairwise Tukey test for {question.capitalize()} question')
    
    # print('\\begin{table}\n', re.sub(r'\%\\caption\{.*\}', f'\\\caption{{Pairwise Tukey test for {question.capitalize()} question}}', tukey_results[question].summary().as_latex_tabular()), '\n\\end{table}\n\n')

In [ ]:
for question, tukey_data in tukey_results.items():
    df = pd.DataFrame(tukey_data.summary())
    df = df.map(lambda x: x.data)
    df.columns = df.iloc[0]
    df = df.iloc[1:]
    df = df.drop(columns='reject')
    df = df[['group1', 'group2', 'meandiff', 'lower', 'upper', 'p-adj']]
    df = df.rename(columns = {
        'group1': 'Label 1',
        'group2': 'Label 2',
        'meandiff': '$\hat{y}_2 - \hat{y}_1$',
        'lower': 'Lower bound',
        'upper': 'Upper bound',
        'p-adj': 'p-value'
    })
    latex = df.to_latex(index=False,
                        column_format='c'*len(df.columns),
                        float_format='%.4f',
                        position='htbp',
                        caption=f'Pairwise Tukey test for {question.capitalize()} question')
    print(latex.replace('\n', '\n\\centering\n', 1))

In [ ]:
topic_analyses = {}

for topic in tqdm(stacked['topic'].unique()):
    print(f'==={topics[topic]}===')
    data = stacked[stacked['topic'] == topic]
    topic_analyses[topic] = {}
    
    for question in ['tone', 'clarity', 'intent']:
        topic_analyses[topic][question] = pairwise_tukeyhsd(endog=data[question], groups=data['label'], alpha=0.05)
    
        print(question, '\n', topic_analyses[topic][question].summary(), '\n\n')
        
        _ = topic_analyses[topic][question].plot_simultaneous(figsize=(5,3), xlabel=f'{question.capitalize()} ranking', ylabel='Label')
        plt.title(f'Pairwise Tukey test for {question.capitalize()} question')

In [ ]:
for topic, topic_results in topic_analyses.items():
    for question, tukey_data in topic_results.items():
        df = pd.DataFrame(tukey_data.summary())
        df = df.map(lambda x: x.data)
        df.columns = df.iloc[0]
        df = df.iloc[1:]
        df = df.drop(columns='reject')
        df = df[['group1', 'group2', 'meandiff', 'lower', 'upper', 'p-adj']]
        df = df.rename(columns = {
            'group1': 'Label 1',
            'group2': 'Label 2',
            'meandiff': '$\hat{y}_2 - \hat{y}_1$',
            'lower': 'Lower bound',
            'upper': 'Upper bound',
            'p-adj': 'p-value'
        })
        
        latex = df.to_latex(index=False,
                            column_format='c'*len(df.columns),
                            float_format='%.4f',
                            position='htbp',
                            caption=f'Pairwise Tukey test for {question.capitalize()} question on {topics[topic].capitalize()} messages')
        print(latex.replace('\n', '\n\\centering\n', 1))

In [ ]:
_, axs = plt.subplots(nrows=3, figsize=(5,10))

for i, question in enumerate(['tone', 'clarity', 'intent']):
    groups = [stacked[stacked['label'] == label][question].values for label in stacked['label'].unique()]
    statistic, p_value = levene(*groups)
    
    # sns.swarmplot(x=question, y='label', data=stacked, hue='label', s=3, ax=axs[i])
    sns.violinplot(x=question, y='label', data=stacked, hue='label', inner='box', cut=0, density_norm='count', alpha=0.5, ax=axs[i], legend=False)
    
    axs[i].set_ylabel('Label')
    axs[i].set_xlabel(f'{question.capitalize()}, (Levene\'s Test: statistic={statistic:.3f}, p={p_value:.3f})')
    
    
plt.tight_layout()
plt.show()

In [ ]:
for t_ind, topic in topics.items():
    _, axs = plt.subplots(nrows=3, figsize=(5,10))
    
    for i, question in enumerate(['tone', 'clarity', 'intent']):
        groups = [stacked[(stacked['label'] == label) & (stacked['topic'] == t_ind)][question].values for label in stacked['label'].unique()]
        statistic, p_value = levene(*groups)
        
        # sns.swarmplot(x=question, y='label', data=stacked, hue='label', s=3, ax=axs[i])
        sns.violinplot(x=question, y='label', data=stacked[stacked['topic'] == t_ind], hue='label', inner='box', cut=0, density_norm='count', alpha=0.5, ax=axs[i], legend=False)
        
        axs[i].set_ylabel('Label')
        axs[i].set_xlabel(f'{question.capitalize()}, (Levene\'s Test: statistic={statistic:.3f}, p={p_value:.3f})')
    
    plt.suptitle(topic)
    plt.tight_layout()
    plt.show()